In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install conda-forge::pdbfixer

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [ ]:
!conda install conda-forge::openmm

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.7.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



In [1]:
import urllib.request
import pdbfixer
from openmm.app import *
from openmm import *
from simtk.unit import *
from pdbfixer import PDBFixer
from openmm.app import Modeller
from openmm.app import StateDataReporter, DCDReporter, CheckpointReporter
from openmm import Platform
import sys
import numpy as np


def download_pdbx(pdb_id):
    """
    Downloads an mmCIF file from RCSB PDB using its PDB ID.
    """
    url = f'https://files.rcsb.org/download/{pdb_id}.cif'
    pdbx_file = f'{pdb_id}.pdbx'

    try:
        print(f"Downloading {pdb_id}.pdbx from RCSB...")
        urllib.request.urlretrieve(url, pdbx_file)
        print(f"Downloaded {pdbx_file}")
    except Exception as e:
        print(f"Failed to download mmCIF file: {e}")
        raise

    return pdbx_file

def load_pdbx_into_pdbfixer(pdbx_file, ignore_missing_residues=True, ignore_terminal_missing_residues=False, ph=7.0):
    """
    Loads the mmCIF (PDBX) file into PDBFixer and prepares the structure.

    Parameters
    ----------
    pdbx_file: str
        Path to the mmCIF file.
    ignore_missing_residues: bool, optional
        If missing residues should be ignored or built.
    ignore_terminal_missing_residues: bool, optional
        If missing residues at the beginning and the end of a chain should be ignored or built.
    ph: float, optional
        pH value used to determine protonation state of residues.

    Returns
    -------
    fixer: pdbfixer.PDBFixer
        Prepared structure.
    """
    fixer = pdbfixer.PDBFixer(str(pdbx_file))
    fixer.removeHeterogens(keepWater=False)  # co-crystallized ligands are unknown to PDBFixer, and removing water
    fixer.findMissingResidues()  # identify missing residues, needed for identification of missing atoms

    # if missing terminal residues shall be ignored, remove them from the dictionary
    if ignore_terminal_missing_residues:
        chains = list(fixer.topology.chains())
        keys = fixer.missingResidues.keys()
        for key in list(keys):
            chain = chains[key[0]]
            if key[1] == 0 or key[1] == len(list(chain.residues())):
                del fixer.missingResidues[key]

    # if all missing residues shall be ignored ignored, clear the dictionary
    if ignore_missing_residues:
        fixer.missingResidues = {}


    fixer.findNonstandardResidues()  # find non-standard residue
    fixer.replaceNonstandardResidues()  # replace non-standard residues with standard one
    fixer.findMissingAtoms()  # find missing heavy atoms
    fixer.addMissingAtoms()  # add missing atoms and residues
    fixer.addMissingHydrogens(ph)  # add missing hydrogens
    return fixer
    return fixer



def prepare_dna(fixer):
    modeller = Modeller(fixer.topology, fixer.positions)
    return modeller



def run_md_simulation(pdb_id):
    """
    Downloads structure in mmCIF format, prepares it, and runs a molecular dynamics simulation.
    """
    # Step 1: Download the mmCIF file from RCSB
    cif_file = download_pdbx(pdb_id)

    # Step 2: Load the mmCIF file into PDBFixer
    fixer = load_pdbx_into_pdbfixer(cif_file)

    # Step 3: Prepare the DNA structure
    modeller= prepare_dna(fixer)

    # Add water molecules (TIP3P model)

    # Load the CHARMM36 force field for nucleic acids
    forcefield = ForceField('amber14/DNA.OL15.xml', 'amber14/tip3pfb.xml')

    modeller.addSolvent(forcefield, padding=1.5 * nanometer)
    print("Done adding solvent")


    # Step 4: Set up integrator and platform
    timestep = 0.002 * picoseconds  # 2 fs timestep
    integrator = LangevinIntegrator(
        300 * kelvin,    # Temperature
        1.0 / picosecond,   # Friction coefficient
        timestep            # Timestep
    )

    #Use CUDA for GPU acceleration
    #platform = Platform.getPlatformByName('CUDA')
    #properties = {'CudaPrecision': 'mixed'}

    # Create a system with constraints but without nonbonded interactions
    system = forcefield.createSystem(
        modeller.topology,
        constraints=HBonds,
        rigidWater=False,
        nonbondedMethod=PME
    )

    # Create a simulation object
    #simulation = Simulation(modeller.topology, system, integrator, platform, properties)
    simulation = Simulation(modeller.topology, system, integrator)
    # Set initial positions
    simulation.context.setPositions(modeller.positions)

    # Step 5: Minimize the energy of the system
    print("Minimizing energy...")
    simulation.minimizeEnergy()

    # Save minimized structure
    positions = simulation.context.getState(getPositions=True).getPositions()
    PDBFile.writeFile(simulation.topology,
                      positions,
                      open(f'{pdb_id}_minimized.pdb', 'w'))

    # Reporters to track the simulation progress
    # Set up reporters
    steps = 100
    write_interval = 1
    log_interval = 1

    simulation.reporters.append(
         XTCReporter(file="trajectory.xtc", enforcePeriodicBox=False, reportInterval=write_interval)
    )

    simulation.reporters.append(
        StateDataReporter(
        sys.stdout,
        log_interval,
        step=True,
        potentialEnergy=True,
        temperature=True,
        progress=True,
        remainingTime=True,
        speed=True,
        totalSteps=steps,
        separator="\t",
        )
    )
    simulation.context.setVelocitiesToTemperature(300 * kelvin)
    simulation.step(steps)
    print("MD simulation complete.")

# Example usage: Running a 2 ns simulation for 7BHO
run_md_simulation('1zew')

Downloaded 1zew.pdbx
Done adding solvent
Minimizing energy...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
1.0%	1	-313119.21179651574	266.0666232012506	0	--
2.0%	2	-306435.55513997027	226.52197673812282	0.716	0:23
3.0%	3	-307063.87823187135	224.56747320415252	0.578	0:29
4.0%	4	-302456.7488241577	195.0387839613906	0.609	0:27
5.0%	5	-293417.1154970618	143.42654112699827	0.58	0:28
6.0%	6	-293841.0099734103	143.7534567514851	0.602	0:26
7.0%	7	-297575.7293014727	163.3886548188672	0.575	0:27
8.0%	8	-293634.2406239344	143.74018074242136	0.592	0:26
9.0%	9	-292397.2383216785	139.01421772793884	0.58	0:27
10.0%	10	-299018.4268928681	175.50520973603676	0.587	0:26
11.0%	11	-299702.0437256884	180.60260741454243	0.574	0:26
12.0%	12	-294540.3022674344	153.96610613048804	0.586	0:25
13.0%	13	-296715.5178800587	165.59647514603336	0.576	0:26
14.0%	14	-300267.38891906384	184.23845735680698	0.583	0:25
15.0%	15	-295373.7797378656	158.2427518295898	0.

In [3]:
from google.colab import files

# Load the trajectory and topology for visualization
pdb_file = '1zew_minimized.pdb'
tr_file = 'trajectory.xtc'

# Download
files.download(pdb_file)
files.download(tr_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>